In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [2]:
path_to_labels = "../../data/labels_mp_0.0005_1000_2017/"
path_to_merged = "../../data/quotes_merged_mp/"
path_to_har = '../../data/HAR/RV_Erfan.csv'

In [3]:
%matplotlib widget

ModuleNotFoundError: No module named 'ipympl'

In [15]:
date = '2017-06-01'
merged_df = pd.read_csv("{}/{}-mt.csv".format(path_to_merged, date))
merged_df.reset_index(level=0, inplace=True)
merged_df['Date-Time'] = pd.to_datetime(merged_df['Date-Time'])
# merged_df.head()

In [16]:
bars_df = pd.read_csv("{}/bars_{}-mt.csv".format(path_to_labels, date))
bars_df['Date-Time'] = pd.to_datetime(bars_df['Date-Time'])

In [17]:
start_time = '2017-06-01 9:00:01.149905303-05:00'
end_time = '2017-06-01 11:25:35.578865996-05:00'

In [18]:
merged_df = merged_df[merged_df['Date-Time']>start_time]
merged_df = merged_df[merged_df['Date-Time']<end_time]
bars_df = bars_df[bars_df['Date-Time']>start_time]
bars_df = bars_df[bars_df['Date-Time']<end_time]
print(merged_df.shape)
print(bars_df.shape)
inds = bars_df.index.to_list()

(181808, 10)
(436, 17)


In [19]:
def plot_barriers(i, df, merged_df, max_ind, pt_level=0.0001, sl_level=0.0001):
#     print(df.loc[i,'Bid Price']*np.array([1+pt_level,1-sl_level]), df.loc[i,'Date-Time'])
#     print(df.loc[i,'Price']*np.array([1+pt_level,1-sl_level]), df.loc[i,'Date-Time'], df.loc[merged_df.loc[i,'Date-Time']])
#     plt.hlines(df.loc[i,'Price']*np.array([1+pt_level,1-sl_level]), xmin=df.loc[i,'Date-Time'], xmax=df.loc[df.loc[i,'end_ind'],'Date-Time'], color='r', linestyles='--')
    c=['k','b','r'][df.loc[i,'long_label']]
#     print(df.loc[i,'label'], c)
    end_ind=min(int(df.loc[i,'end_ind']),max_ind)
    plt.hlines(df.loc[i,'Ask Price']*np.array([1+df.loc[i,'dailyVolatility']*pt_level,1-df.loc[i,'dailyVolatility']*sl_level]), xmin=df.loc[i,'Date-Time'], xmax=merged_df.loc[end_ind,'Date-Time'], color=c, linestyles='--')
    plt.vlines(x = [df.loc[i,'Date-Time'], merged_df.loc[end_ind,'Date-Time']], ymin=df.loc[i,'Ask Price']*(1-df.loc[i,'dailyVolatility']*sl_level), ymax=df.loc[i,'Ask Price']*(1+df.loc[i,'dailyVolatility']*pt_level), color=c, linestyles='--')
    plt.text(df.loc[i,'Date-Time'],df.loc[i,'Bid Price']+2,i )
# plot_barriers(i, bars_df)

In [20]:
# %matplotlib inline

In [21]:
%matplotlib widget

In [22]:
fig, ax = plt.subplots()

# fig.canvas.toolbar_visible = True
# fig.canvas.header_visible = False
# fig.canvas.resizable = True

merged_df.plot(x='Date-Time', y='Bid Price', style='.-', color='b', figsize=(12,6),ax=ax)
# merged_df.plot(x='Date-Time', y='Ask Price', style='.-', color='r', figsize=(12,6), ax=ax)
# merged_df.plot(y='Price', style='.', figsize=(16,12))
plt.xlabel("Time")
plt.ylabel("Price")
max_ind = merged_df.index.max()
for i in [inds[x] for x in range(0,len(inds),30)]:
#     print(i)
    plot_barriers(i, bars_df, merged_df,max_ind , pt_level=0.0003, sl_level=0.0003)
plt.show();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
bars_df.loc[584,:]

original_index                                  277576
Date-Time          2017-06-01 10:46:50.142220892-05:00
Bid Price                                      2416.75
Ask Price                                         2417
Bid Size                                           596
Ask Size                                           288
Seq. No.                                       7440444
Exch Time                           15:46:50.133000000
#RIC                                              ESM7
dailyVolatility                                0.25142
pt_long_ind                                     278722
sl_long_ind                                        NaN
pt_short_ind                                       NaN
sl_short_ind                                    278723
end_ind                                         289383
long_label                                           1
short_label                                         -1
Name: 584, dtype: object